In [174]:
from spacy.tokens import DocBin
import pandas as pd
import string
import spacy
import gensim
from gensim.utils import simple_preprocess

In [176]:
train_data = pd.read_excel("change_training.xlsx")
valid_data = pd.read_excel("change_training.xlsx")

In [133]:
train_data = list(train_data.itertuples(index = False, name = None))
valid_data = list(valid_data.itertuples(index = False, name = None))

In [134]:
def remove_punctuation(input_string):
        translator = str.maketrans("","", string.punctuation)
        return input_string.translate(translator)

def note_replacer(note_value):
    clean_note = ""  
    note_value = remove_punctuation(note_value)    
    for char in note_value:
        if char.isalpha():
            clean_note += char.lower()
        else:
            clean_note += char
    return clean_note

In [135]:
for value in train_data:
    index = train_data.index(value)
    value = (note_replacer(value[0]), value[1])
    train_data[index] = value

for value in valid_data:
    index = valid_data.index(value)
    value = (note_replacer(value[0]), value[1])
    valid_data[index] = value

In [152]:
def make_docs(data):
    docs = []
    for doc, label in nlp.pipe(data, as_tuples = True):
        if label == "increase":
            doc.cats["increase"] = 1
            doc.cats["decrease"] = 0
            doc.cats["erratic"] = 0
        elif label == "decrease":
            doc.cats["increase"] = 0
            doc.cats["decrease"] = 1
            doc.cats["erratic"] = 0
        elif label == "erratic":
            doc.cats["increase"] = 0
            doc.cats["decrease"] = 0
            doc.cats["erratic"] = 1
        docs.append(doc)
    return(docs)

In [154]:
nlp = spacy.load("en_core_web_lg")

In [155]:
num_texts = 1000

train_docs = make_docs(train_data[:num_texts])
doc_bin = DocBin(docs=train_docs)
doc_bin.to_disk("./data/train.spacy")

valid_docs = make_docs(valid_data[:num_texts])
doc_bin = DocBin(docs=valid_docs)
doc_bin.to_disk("./data/valid.spacy")